In [48]:
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import openpyxl.workbook
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill
import os

class Aplicativo():
    def __init__(self, raiz):
        self.raiz = raiz
        self.raiz.title("Formatação Condicional em Tabela")
        self.raiz.geometry("950x650")
        
        self.dados_excel = None
        self.caminho_arquivo = None
        
        self.container_principal = tk.Frame(raiz,bg='#f7f7f7')
        self.container_principal.pack(fill=tk.BOTH,
                                      expand=True,
                                      padx=20,
                                      pady=20)
        
        self.titulo = tk.Label(self.container_principal,
                               text='Formatação Condicional em Tabela',
                               font=("Arial",20,"bold"),
                               bg='#f7f7f7')
        self.titulo.grid(row=0,columnspan=4,pady=10)

        self.container_arquivo = tk.Frame(self.container_principal,bg='#f7f7f7')
        self.container_arquivo.grid(row=1,columnspan=4,pady=10)

        self.botao_carregar_arquivo = tk.Button(self.container_arquivo,
                                                text='Escolher Arquivo',
                                                command=self.carregar_arquivo,
                                                bg="#4caf50",
                                                fg='white',
                                                font=("Arial",12,'bold'))
        self.botao_carregar_arquivo.pack(side=tk.LEFT,padx=5)

        self.rotulo_nome_arquivo = tk.Label(self.container_arquivo,
                                            text="Nenhum arquivo escolhido",
                                            bg='#f7f7f7',
                                            font=('Arial',12))
        self.rotulo_nome_arquivo.pack(side=tk.LEFT,padx=5)

        self.menu_selecao = tk.Frame(self.container_principal,
                                     bg='#f7f7f7')
        self.menu_selecao.grid(row=2,columnspan=4,pady=10)

        self.rotulo_selecao_coluna = tk.Label(self.menu_selecao,
                                              text='Selecione a Coluna: ',
                                              bg='#f7f7f7',
                                              font=("Arial",12))
        self.rotulo_selecao_coluna.grid(row=0,column=0,padx=5,sticky="e")

        self.selecao_coluna = ttk.Combobox(self.menu_selecao,
                                           font=("Arial",12))
        self.selecao_coluna.grid(row=0,column=1,padx=5)

        self.rotulo_condicao = tk.Label(self.menu_selecao,
                                        text='Condição: ',
                                        bg='#f7f7f7',
                                        font=("Arial",12))
        self.rotulo_condicao.grid(row=0,column=2,padx=5,sticky='e')

        self.selecao_condicao = ttk.Combobox(self.menu_selecao,
                                             values=[
                                                 "Maior que",
                                                 "Menor que",
                                                 "Igual a",
                                                 "Entre",
                                                 "Texto que Contém",
                                                 "Uma Data de Ocorre...",
                                                 "Valores Duplicados"],
                                             font=("Arial",12))
        self.selecao_condicao.grid(row=0,column=3,padx=5)
        self.selecao_condicao.bind("<<ComboboxSelected>>",self.condicao_selecionada)

        self.rotulo_valor1 = tk.Label(self.menu_selecao,
                                      text='Valor 1: ',
                                      bg="#f7f7f7",
                                      font=("Arial",12))
        self.rotulo_valor1.grid(row=1,column=0,padx=5,sticky='e')

        self.campo_valor1 = tk.Entry(self.menu_selecao,
                                     font=("Arial",12))
        self.campo_valor1.grid(row=1,column=1,padx=5)

        self.rotulo_valor2 = tk.Label(self.menu_selecao,
                                      text='Valor 2: ',
                                      bg="#f7f7f7",
                                      font=("Arial",12)) 
               
        self.campo_valor2 = tk.Entry(self.menu_selecao,
                                     font=("Arial",12))
        
        self.caixa_mostrar_filtrados = tk.BooleanVar()
        
        self.checkbox_filtrados = tk.Checkbutton(self.menu_selecao,
                                                 text="Mostrar apenas filtrados",
                                                 variable=self.caixa_mostrar_filtrados,
                                                 bg='#f7f7f7',
                                                 font=("Arial",12))
        self.checkbox_filtrados.grid(row=2,column=0,columnspan=2,padx=5) 

        self.aplicar_formatacao = tk.Button(self.menu_selecao,
                                            text="Aplicar Formatação",
                                            command=self.aplicar_formatacao_condicional,
                                            bg='#008cba',
                                            fg='white',
                                            font=("Arial",12))
        self.aplicar_formatacao.grid(row=2,column=2,padx=5)

        self.exportar_dados = tk.Button(self.menu_selecao,
                                            text="Exportar Dados Filtrados",
                                            command=self.exportar_dados_filtrados,
                                            bg='#f44336',
                                            fg='white',
                                            font=("Arial",12))
        self.exportar_dados.grid(row=2,column=3,padx=5)

        self.container_tabela = tk.Frame(self.container_principal)
        self.container_tabela.grid(row=3,
                                   column=0,
                                   columnspan=4,
                                   padx=10,
                                   sticky='nsew')
        
        self.container_principal.grid_rowconfigure(3,weight=1)
        
        self.container_principal.grid_columnconfigure(0,weight=1)

        
    def carregar_arquivo(self):
        self.caminho_arquivo = filedialog.askopenfilename(filetypes=[("Excel files","*.xlsx *.xls")])
        if self.caminho_arquivo:
            self.rotulo_nome_arquivo.config(text=os.path.basename(self.caminho_arquivo))
            self.dados_excel = pd.read_excel(self.caminho_arquivo)
            self.preencher_tabela()
            self.preencher_selecao_coluna()

    def preencher_tabela(self):
        for widget in self.container_tabela.winfo_children():
            widget.destroy()

        self.tabela = ttk.Treeview(self.container_tabela,
                                   columns=list(self.dados_excel.columns),
                                   show='headings')
        self.tabela.pack(fill=tk.BOTH,expand=True)

        for col in self.dados_excel.columns:
            self.tabela.heading(col, text=col)
            self.tabela.column(col,width=100)

        for idx,row in self.dados_excel.iterrows():
            self.tabela.insert('', tk.END, values=list(row),iid=str(idx))

    def preencher_selecao_coluna(self):
        self.selecao_coluna['values'] = list(self.dados_excel.columns)

        self.selecao_coluna.current(0)

    def condicao_selecionada(self,event):
        condicao = self.selecao_condicao.get()

        if condicao == "Entre":
            self.campo_valor2.grid(row=1,column=3,padx=5)
            self.rotulo_valor2.grid(row=1,column=2,padx=5,sticky='e')
        else:
            self.campo_valor2.grid_remove()
            self.rotulo_valor2.grid_remove()

    def aplicar_formatacao_condicional(self):
        self.preencher_tabela()

        coluna_selecionada = self.selecao_coluna.get()
        
        condicao = self.selecao_condicao.get()

        valor1 = self.campo_valor1.get()
        valor2 = self.campo_valor2.get()

        serie_coluna = self.dados_excel[coluna_selecionada]

        itens_para_oculta = []

        for item in self.tabela.get_children():
            idx = int(item)
            valor = serie_coluna.iloc[idx]
            aplicar_cor = False
            for item in self.tabela.get_children():
                idx = int(item)
                valor = serie_coluna.iloc[idx]
                aplicar_cor = False

                try:
                    if condicao == "Maior que":
                        aplicar_cor = pd.to_numeric(valor,errors='coerce') > pd.to_numeric(valor1,errors='coerce')
                    elif condicao == "Menor que":
                        aplicar_cor = pd.to_numeric(valor,errors='coerce') < pd.to_numeric(valor1,errors='coerce')
                    elif condicao == "Igual a":
                        aplicar_cor = (str(valor)==valor1)
                    elif condicao == "Entre":
                        aplicar_cor = pd.to_numeric(valor1,errors='coerce') <= pd.to_numeric(valor,errors='coerce') <= pd.to_numeric(valor2,errors='coerce')
                    elif condicao == "Texto que Contém":
                        aplicar_cor = valor1 in str(valor)
                    elif condicao == "Uma Data de Ocorre...":
                        aplicar_cor = pd.to_datetime(valor,errors='coerce',dayfirst=True) == pd.to_datetime(valor1,errors='coerce',dayfirst=True)
                    elif condicao == "Valores Duplicados":
                        aplicar_cor = serie_coluna.duplicated(keep=False).iloc[idx]
                    
                except ValueError:
                    continue

                if aplicar_cor:
                    self.tabela.item(item,tags=("hightlight"))

                else:
                    itens_para_oculta.append(item)

                if self.caixa_mostrar_filtrados.get():
                    for item in itens_para_oculta:
                        self.tabela.detach(item)
                else:
                    for item in self.tabela.get_children():
                        self.tabela.reattach(item,'',tk.END)

            self.tabela.tag_configure("hightlight",background="yellow")

    def exportar_dados_filtrados(self):
        if self.dados_excel is not None:
            caminho_exportacao = filedialog.asksaveasfilename(defaultextension=".xlsx", 
                                                              filetypes=[("Excel files", "*.xlsx *.xls")])
            if caminho_exportacao:
                workbook = openpyxl.Workbook()
                planilha = workbook.active

                for col_num,col_nome in enumerate(self.dados_excel,1):
                    planilha.cell(row=1,column=col_num).value = col_nome
                
                for row_num,item in enumerate(self.tabela.get_children(),2):
                    valores = self.tabela.item(item,'values')
                    for col_num,valor in enumerate(valores,1):
                        celula = planilha.cell(row=row_num,column=col_num)
                        celula.value = valor

                        if "hightlight" in self.tabela.item(item, 'tags'):
                            
                           celula.fill = PatternFill(start_color="FFFF00", 
                                                      end_color="FFFF00", 
                                                      fill_type="solid")
                            
                workbook.save(caminho_exportacao)
                messagebox.showinfo("Exportação","Dados exportados com sucesso!")   

raiz = tk.Tk()
aplicaativo = Aplicativo(raiz)

raiz.mainloop()